# Neural Network Model

# to do:
Add VIX and VIX return in the correlation check 
manage names (spy volumne, spy return spy close, etc

In [280]:
# Import appropriate modules
import pandas as pd
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import datetime
import numpy as np
import yfinance as yf
from datetime import datetime
from pandas.tseries.offsets import DateOffset
import hvplot
import hvplot.pandas
import matplotlib.pyplot as plt
from vix_functions import garch_fit_and_predict, correlation_filter, retrieve_yahoo_close, retrieve_yahoo_volume 

# Generation of the Features Matrix X

In [80]:
# Ticker List: VIX must be in first position
ticker_list= ["^VIX", "spy", "DX-Y.NYB", "tlt", "ief", "gld", "slv", "CL=F", "USDJPY=X", "FXI", "EZU", "EEM", "EFA"]

In [272]:
# Inclusion of security levels X1
def retrieve_close(close_prices_dict, ticker_list):
    for ticker in ticker_list:
        close_price = retrieve_yahoo_close(ticker)
        close_prices_dict[ticker] = close_price
    return close_prices_dict

close_prices_dict = {}
close_prices_dict = retrieve_close(close_prices_dict, ticker_list)
print("Completed retrieve of close prices")

close_prices_df= pd.DataFrame(close_prices_dict)

close_prices_component_df = correlation_filter(close_prices_df, min_corr=0.20, key_column='^VIX', eliminate_first_column=True)

X1=close_prices_component_df
vix=close_prices_df['^VIX']
vix_ret=close_prices_df['^VIX'].pct_change()
VIX=pd.DataFrame([vix, vix_ret]).T
VIX.columns=['VIX','VIX_ret']

X1=pd.concat([VIX,close_prices_component_df], axis=1)

Processing Close ^VIX
Processing Close spy
Processing Close DX-Y.NYB
Processing Close tlt
Processing Close ief
Processing Close gld
Processing Close slv
Processing Close CL=F
Processing Close USDJPY=X
Processing Close FXI
Processing Close EZU
Processing Close EEM
Processing Close EFA
Completed retrieve of close prices


In [250]:
# Inclusion of security returns X2
# Include returns that are correlated more than 0.20 with the Vix return

security_returns_df= close_prices_df.pct_change()
security_returns_component_df = correlation_filter(security_returns_df, min_corr=0.20, key_column='^VIX', eliminate_first_column=True )

X2=security_returns_component_df


#### OBSERVATION: It would be good to include VIX level and Return and check on both in case correlations changes in the future. As of now with some variables I check with VIX level, and others I check VIX return

In [251]:
# inclusion of security volume X3
volume_list = ticker_list[1:len(ticker_list)]

def retrieve_volume(volume_dict, volume_list):
    for ticker in volume_list:        
        volume = retrieve_yahoo_volume(ticker)
        volume_dict[ticker] = volume
    return volume_dict

volume_dict = {}
volume_dict = retrieve_volume(volume_dict, volume_list)
print("Completed retrieve of volume")

volume_df= pd.DataFrame(volume_dict)
volume_df_with_vix=pd.concat([vix, volume_df], axis=1)
#print(volume_df_with_vix.corr())

volume_component_df = correlation_filter(volume_df_with_vix, min_corr=0.20, key_column='^VIX', eliminate_first_column=True )
X3=volume_component_df

Processing Volume spy
Processing Volume DX-Y.NYB
Processing Volume tlt
Processing Volume ief
Processing Volume gld
Processing Volume slv
Processing Volume CL=F
Processing Volume USDJPY=X
Processing Volume FXI
Processing Volume EZU
Processing Volume EEM
Processing Volume EFA
Completed retrieve of volume


In [252]:
# Inclusion of GARCH series X4
garch_series=pd.DataFrame()

for ticker in ticker_list:
        garch_series[ticker]=garch_fit_and_predict(security_returns_df[ticker], horizon=1, p=1, q=1, o=1)

X4=garch_series
#X4


In [253]:
# Inclusion of return squares in X5

returns_squared_df_no_vix= security_returns_df.drop(columns='^VIX')**2
returns_squared_and_vix_level_df=pd.concat([vix,returns_squared_df_no_vix], axis=1)
returns_squared_component_df = correlation_filter(returns_squared_and_vix_level_df, min_corr=0.20, key_column='^VIX', eliminate_first_column=True)

X5=returns_squared_component_df
X5

,spy,DX-Y.NYB,tlt,ief,gld,slv,FXI,EZU,EEM,EFA
Date,,,,,,,,,,
2007-07-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-07-03,0.000013,0.000000e+00,3.717331e-05,0.000009,0.000019,0.000020,0.000439,0.000042,6.347566e-05,2.265476e-05
2007-07-04,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
2007-07-05,0.000001,3.396638e-06,1.255927e-04,0.000031,0.000024,0.000029,0.000012,0.000027,1.964648e-05,2.480378e-05
2007-07-06,0.000028,9.626279e-07,1.645825e-05,0.000006,0.000070,0.000298,0.000630,0.000118,2.936444e-04,5.187043e-05
...,...,...,...,...,...,...,...,...,...,...
2021-09-27,0.000008,2.869577e-07,1.351201e-05,0.000007,0.000004,0.000123,0.000207,0.000001,4.223255e-05,3.877590e-07
2021-09-28,0.000406,1.744296e-05,2.426398e-04,0.000015,0.000094,0.000058,0.000002,0.000569,1.667539e-04,4.755389e-04
2021-09-29,0.000003,3.695060e-05,3.010235e-06,0.000001,0.000020,0.001595,0.000052,0.000024,8.679118e-05,5.257838e-06


### Inclusion of Google Trends

In [254]:
# Upload of csv files
keywords=['liquidity', 'Jobless_claims']
google_trends_df=pd.DataFrame()

for keyword in keywords:
    file_path=f"./Resources/{keyword}.csv"
    print(file_path)
    trend=pd.read_csv(Path(file_path),
                      index_col= 'Date', 
                      parse_dates= True,
                      infer_datetime_format=True)
    google_trends_df=pd.concat([google_trends_df, trend], axis=1)

google_trends_df

./Resources/liquidity.csv
./Resources/Jobless_claims.csv


,liquidity: (United States),Jobless claims: (United States)
Date,,
2016-10-09,31,0
2016-10-16,54,0
2016-10-23,42,0
2016-10-30,38,0
2016-11-06,32,0
...,...,...
2021-09-05,31,0
2021-09-12,43,0
2021-09-19,37,9


In [255]:
# We will interpolate with degree 2 in case of missing values
google_trends_df.iloc[0,:]=0

vix_google_trends_df=pd.concat([vix, google_trends_df], axis=1)
vix_google_trends_df.iloc[0,1:]=0
vix_google_trends_df.head(20)

vix_google_trends_df=vix_google_trends_df.interpolate(method="polynomial", order=2)

In [256]:
# Filtering by correlation X6


google_trends_component_df = correlation_filter(
                                vix_google_trends_df, 
                                min_corr=0.05, 
                                key_column='^VIX', 
                                eliminate_first_column=True)

X6=google_trends_component_df

# INCLUSION OF LAGGED SERIES

In [10]:
# Inclusion of different lags of data from the X -- up to n_lag



# GENERATION OF THE FEATURE MATRIX **X**

In [344]:
# Concatenation of all sources of data
XY=pd.concat([X1, X2, X3, X4, X5, X6], axis=1)
#print("X without interpolation")
#print(X)

# Fill missing values with interpolation
XY=XY.interpolate(method="polynomial", order=2)
#print(XY)
#print("XY with interpolation")

# Drop first or last rows if they are NA
XY=XY.dropna()
#print("XY after drop Nan")
print(XY.columns)

Index(['VIX', 'VIX_ret', 'spy', 'ief', 'USDJPY=X', 'FXI', 'EZU', 'EEM', 'EFA',
       'spy', 'tlt', 'ief', 'FXI', 'EZU', 'EEM', 'EFA', 'spy', 'gld', 'FXI',
       'EEM', 'EFA', '^VIX', 'spy', 'DX-Y.NYB', 'tlt', 'ief', 'gld', 'slv',
       'CL=F', 'USDJPY=X', 'FXI', 'EZU', 'EEM', 'EFA', 'spy', 'DX-Y.NYB',
       'tlt', 'ief', 'gld', 'slv', 'FXI', 'EZU', 'EEM', 'EFA',
       'liquidity: (United States)', 'Jobless claims: (United States)'],
      dtype='object')


In [345]:
# Set the Signal column
vix_ret=vix.pct_change()
XY["Signal"] = 0.0

XY.loc[(XY['VIX_ret'] >= 0.036), 'Signal'] = 1

# # Generate the trading signals 1 (entry) or -1 (exit)
# # where 1 is when the ^VIX is greater than 3.6%.
# # where 0 is when the ^VIX  is less than 3.6%.
#for index, row in XY.iterrows():
#    if row["VIX_ret"] >= 0.036:
#        XY.loc[index, "Signal"] = 1.0

# Review the DataFrame
print(XY["Signal"].head()    )    
XY["Signal"].value_counts()

Date
2007-07-03    0.0
2007-07-04    0.0
2007-07-05    1.0
2007-07-06    0.0
2007-07-09    0.0
Name: Signal, dtype: float64


0.0    2996
1.0     983
Name: Signal, dtype: int64

In [326]:
vix_ret=vix.pct_change()
vix_ret[vix_ret>=0.036].index

DatetimeIndex(['2007-07-05', '2007-07-10', '2007-07-20', '2007-07-24',
               '2007-07-26', '2007-07-27', '2007-07-31', '2007-08-03',
               '2007-08-09', '2007-08-10',
               ...
               '2021-08-17', '2021-08-18', '2021-08-26', '2021-09-07',
               '2021-09-09', '2021-09-10', '2021-09-17', '2021-09-20',
               '2021-09-27', '2021-09-28'],
              dtype='datetime64[ns]', name='Date', length=903, freq=None)

In [346]:
# Define the target set y using the Signal column
y = XY["Signal"]
# Display a sample of y
y

Date
2007-07-03    0.0
2007-07-04    0.0
2007-07-05    1.0
2007-07-06    0.0
2007-07-09    0.0
             ... 
2021-09-27    1.0
2021-09-28    1.0
2021-09-29    0.0
2021-09-30    0.0
2021-10-01    0.0
Name: Signal, Length: 3979, dtype: float64

In [347]:
# Define features set X by selecting all columns but Signal
X = XY.drop(columns=["Signal"]).shift().dropna()


# Review the features DataFrame
X.head()

,VIX,VIX_ret,spy,ief,USDJPY=X,FXI,EZU,EEM,EFA,spy,...,tlt,ief,gld,slv,FXI,EZU,EEM,EFA,liquidity: (United States),Jobless claims: (United States)
Date,,,,,,,,,,,,,,,,,,,,,
2007-07-04,14.920000,-0.031169,114.571579,57.929962,122.470001,32.352470,39.293316,33.974319,53.867046,0.003623,...,0.000037,0.000009,0.000019,0.000020,0.000439,0.000042,0.000063,0.000023,-9.351484,-0.000046
2007-07-05,15.275253,0.000000,114.463078,57.763530,122.660004,32.241711,39.148205,33.981361,53.696419,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-18.697444,-0.000093
2007-07-06,15.480000,0.037533,114.451218,57.606781,122.949997,32.238968,39.090733,34.124908,53.598793,-0.001050,...,0.000126,0.000031,0.000024,0.000029,0.000012,0.000027,0.000020,0.000025,-28.037881,-0.000139
2007-07-09,14.720000,-0.049096,115.052856,57.470295,123.360001,33.047962,39.515484,34.709675,53.984821,0.005257,...,0.000016,0.000006,0.000070,0.000298,0.000630,0.000118,0.000294,0.000052,-37.372794,-0.000185
2007-07-10,15.160000,0.029891,115.143105,57.678589,123.370003,33.504383,39.688667,35.045982,54.115658,0.000785,...,0.000013,0.000013,0.000042,0.000000,0.000191,0.000019,0.000094,0.000006,-65.344391,-0.000324


In [321]:
X.corr()

,VIX,VIX_ret,spy,ief,USDJPY=X,FXI,EZU,EEM,EFA,spy,...,tlt,ief,gld,slv,FXI,EZU,EEM,EFA,liquidity: (United States),Jobless claims: (United States)
VIX,1.000000,0.116827,-0.248277,-0.216504,-0.241448,-0.358406,-0.404144,-0.438938,-0.441902,-0.152124,...,0.365071,0.334268,0.257706,0.272422,0.424691,0.469433,0.427876,0.464775,-0.094912,0.303630
VIX_ret,0.116827,1.000000,0.011708,0.010207,0.020049,-0.002011,0.004183,-0.006372,0.006289,-0.705401,...,0.097539,0.128269,0.091212,0.091664,0.054566,0.168131,0.070482,0.151711,0.017592,-0.008727
spy,-0.248277,0.011708,1.000000,0.871361,0.527559,0.756637,0.825404,0.803808,0.928858,0.029524,...,-0.072727,-0.160742,-0.126465,-0.082098,-0.149607,-0.143064,-0.129129,-0.128770,0.748454,0.309849
ief,-0.216504,0.010207,0.871361,1.000000,0.351873,0.517905,0.515256,0.596555,0.717908,0.020812,...,0.000997,-0.140335,-0.119799,-0.062644,-0.190795,-0.103709,-0.141539,-0.108136,0.545088,0.375171
USDJPY=X,-0.241448,0.020049,0.527559,0.351873,1.000000,0.485745,0.622887,0.249180,0.605484,-0.005745,...,-0.089195,-0.090345,-0.060806,-0.093705,-0.018473,-0.115820,-0.040524,-0.077225,0.749725,0.133179
FXI,-0.358406,-0.002011,0.756637,0.517905,0.485745,1.000000,0.860996,0.885610,0.854559,0.057751,...,-0.118847,-0.165599,-0.160506,-0.110261,-0.185094,-0.216751,-0.192705,-0.202516,0.655781,0.193402
EZU,-0.404144,0.004183,0.825404,0.515256,0.622887,0.860996,1.000000,0.857397,0.959246,0.037361,...,-0.160357,-0.180516,-0.129703,-0.108251,-0.141058,-0.220380,-0.159025,-0.191359,0.720835,0.138107
EEM,-0.438938,-0.006372,0.803808,0.596555,0.249180,0.885610,0.857397,1.000000,0.887817,0.061181,...,-0.137553,-0.195581,-0.164685,-0.104701,-0.233268,-0.234240,-0.224447,-0.227740,0.467562,0.150204
EFA,-0.441902,0.006289,0.928858,0.717908,0.605484,0.854559,0.959246,0.887817,1.000000,0.040320,...,-0.143013,-0.201280,-0.153054,-0.121055,-0.187461,-0.224165,-0.184528,-0.202589,0.719743,0.193886
spy,-0.152124,-0.705401,0.029524,0.020812,-0.005745,0.057751,0.037361,0.061181,0.040320,1.000000,...,-0.190241,-0.152920,-0.067616,-0.105626,0.066593,-0.079797,0.046599,-0.040890,-0.001143,0.009043


In [348]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
training_begin = X.index.min()
training_end = X.index.min() + DateOffset(months=100)

X_train = X.loc[training_begin:training_end]
y_train = y.loc[training_begin:training_end]

X_test = X.loc[training_end+DateOffset(days=1):]
y_test = y.loc[training_end+DateOffset(days=1):]


In [349]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler = scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


In [350]:
# Define the the number of inputs (features) to the model
number_input_features = X.shape[1]

# Review the number of features
number_input_features

46

In [351]:
# Define the number of neurons in the output layer
number_output_neurons = 1

In [352]:
# Define the number of hidden nodes for the first hidden layer - use the square root of the number of input features times number of output neurons
hidden_nodes_layer1 =  int(np.ceil(np.sqrt(number_input_features * number_output_neurons)))

# Review the number hidden nodes in the first layer
hidden_nodes_layer1


7

In [353]:
# Define the number of hidden nodes for the second hidden layer - hidden layer neurons divided by 2
hidden_nodes_layer2 =  int(np.floor(hidden_nodes_layer1/2))

# Review the number hidden nodes in the second layer
hidden_nodes_layer2

3

In [354]:
# Create the Sequential model instance
nn = Sequential()

In [355]:
# Add the first hidden layer
nn.add(Dense(input_dim=number_input_features, units=hidden_nodes_layer1, activation="relu"))

In [356]:
# Add the second hidden layer
nn.add(Dense(units=hidden_nodes_layer2, activation="relu"))

In [357]:
# Add the output layer to the model specifying the number of output neurons and activation function
nn.add(Dense(units=number_output_neurons, activation="sigmoid"))

In [358]:
# Display the Sequential model summary
nn.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 7)                 329       
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 24        
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 4         
Total params: 357
Trainable params: 357
Non-trainable params: 0
_________________________________________________________________


In [359]:
# Compile the Sequential model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [360]:
# Fit the model using 50 epochs and the training data
model_1 = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
68/68 [==============================] - 1s 2ms/step - loss: 0.6849 - accuracy: 0.6002
Epoch 2/50
68/68 [==============================] - 0s 2ms/step - loss: 0.6377 - accuracy: 0.7321
Epoch 3/50
68/68 [==============================] - 0s 2ms/step - loss: 0.6030 - accuracy: 0.7537
Epoch 4/50
68/68 [==============================] - 0s 2ms/step - loss: 0.5744 - accuracy: 0.7578
Epoch 5/50
68/68 [==============================] - 0s 2ms/step - loss: 0.5630 - accuracy: 0.7551
Epoch 6/50
68/68 [==============================] - 0s 2ms/step - loss: 0.5569 - accuracy: 0.7564
Epoch 7/50
68/68 [==============================] - 0s 2ms/step - loss: 0.5538 - accuracy: 0.7564
Epoch 8/50
68/68 [==============================] - 0s 2ms/step - loss: 0.5515 - accuracy: 0.7564
Epoch 9/50
68/68 [==============================] - 0s 2ms/step - loss: 0.5497 - accuracy: 0.7569
Epoch 10/50
68/68 [==============================] - 0s 2ms/step - loss: 0.5476 - accuracy: 0.7564
Epoch 11/50
68/68 [

In [361]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

57/57 - 0s - loss: 7.3065 - accuracy: 0.6876
Loss: 7.306501388549805, Accuracy: 0.6875693798065186
